In [ ]:
from selenium import webdriver
import pandas as pd
import time
import random
import getpass # 実行ユーザーを取得するために使用
import requests
import json
from selenium.webdriver.support.ui import Select # Selectタグが扱えるエレメントに変化させる為の関数を呼び出す
from selenium.webdriver.common.action_chains import ActionChains

In [ ]:
def login():
    driver.get('https://www.mercari.com/jp/login/')
    f=open('mercari.txt','a')
    f.write("mercariにアクセスしました\n")
    f.close()
    time.sleep(1)

    #メアドとパスワードを入力
    driver.find_element_by_name('email').send_keys('')
    time.sleep(1)
    driver.find_element_by_name('password').send_keys('')
    time.sleep(1)

    #recaptureのtextareaとログインボタンをウインドウ内に表示するために最下部にスクロール
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    #recapture対策
    driver.execute_script('var element=document.getElementById("g-recaptcha-response"); element.style.display="";')
    service_key = '' # 2captchaのservice key 
    google_site_key = '6LfZGCYTAAAAAHoK-s7Lg5OdYg5Fq4susvALmZoc' # reCAPTCHAのdata-sitekey
    pageurl = 'https://www.mercari.com/jp/login/' #recaptureのあるサイトurl
    url = "http://2captcha.com/in.php?key=" + service_key + "&method=userrecaptcha&googlekey=" + google_site_key + "&pageurl=" + pageurl 
    resp = requests.get(url) 
    if resp.text[0:2] != 'OK': 
        quit('Service error. Error code:' + resp.text) 
    captcha_id = resp.text[3:]
    fetch_url = "http://2captcha.com/res.php?key="+ service_key + "&action=get&id=" + captcha_id
    for i in range(1, 10):
        time.sleep(5) # wait 5 sec.
        resp = requests.get(fetch_url)
        if resp.text[0:2] == 'OK':
            break
    print('Google response token: ', resp.text[3:])
    driver.find_element_by_id('g-recaptcha-response').send_keys(resp.text[3:])# textareaへトークン入力
    time.sleep(2)
    #ログインボタンを押す（textareaが邪魔でボタンを押せないのでクリック位置を下に1ずらしている
    target=driver.find_element_by_xpath('/html/body/div[1]/main/div/form/div/button')
    actions=ActionChains(driver)
    actions.move_to_element(target)
    actions.move_by_offset(0,1)#targetから下に1マウスカーソル移動
    print(target.location)
    actions.click()
    actions.perform()#実行
    time.sleep(random.randint(2,5))
    f=open('mercari.txt','a')
    f.write("mercariにログインしました\n")
    f.close()

In [ ]:
def delete():
    #出品一覧ページ
    mypage=driver.find_element_by_class_name("mypage-nav-list").find_elements_by_tag_name('a')[1].get_attribute("href")
    driver.get(mypage)
    #出品個別ページ
    driver.find_element_by_class_name('mypage-tab-container').find_elements_by_class_name('mypage-item-link')[0].click()
    #削除ボタン
    driver.find_element_by_css_selector('body > div > main > div.l-content > div.listing-item-change-box > button').click()
    #削除確認ボタン
    driver.find_element_by_css_selector('body > div > main > div.l-content > aside > div > div.clearfix > form > button').click()

In [ ]:
driver = webdriver.Chrome()#指定の位置にwebdriverのpathを書いてください　例）/Users/ユーザー名/chromedriver
time.sleep(5)
login()
time.sleep(5)
delete()
time.sleep(5)
driver.close()